# IBM Data Science Capstone notebook
Hans-Martin Duringhof

<I>This notebook is going to be used for the capstone project for IBM's Applied Data Science Capstone Project course on Coursera</I>

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## <U>Part 1: Segmenting and Clustering Neighborhoods in Toronto</U>

First, the following code scrapes the wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M to create a Pandas dataframe with information from the shown table:

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df=pd.read_html(url, header=0)[0]
df.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [3]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Ignore cells with a borough that is <B>Not assigned</B>:

In [4]:
df = df[df.Borough !='Not assigned']

In [5]:
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Combine multiple neighborhoods that share the same postal code:

In [6]:
df = df.groupby(['PostalCode', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)

In [7]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In those cases where "Neighbourhood" has the value <B>Not assigned</B> the neighbourhood name will be defined the same as "Boroug

In [8]:
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df.Borough

In [9]:
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
df.shape

(103, 3)